# `smolagents` + `simpler.grants.gov` Integration Demo

This notebook demonstrates using the `simpler.grants.gov` tools from `gofannon` to create an agent with `smolagents` that can help you find grants in the U.S.

## Install Dependencies

`simpler.grants.gov` doesn't require any extras, but `smolagents` does so we will install `gofannon` with the `smolagents` extra.

In [2]:
!pip install smolagents git+https://github.com/rawkintrevo/gofannon@297 --quiet

# !pip install gofannon[smolagents] --quiet #uncomment before PR

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 37.9 MB/s eta 0:00:00


## Setting the API Key

Currently API Keys must be requested directly from `simpler.grants.gov` as their API is still in Alpha.

In [3]:
from google.colab import userdata
simpler_grants_api_key = userdata.get('simpler-grants-gov')

## Creating Tools

In the following two blocks, we will import all of our related tools, then export them all to smolagents, so that the smolagents agent can utilize them.

In [4]:
from gofannon.simpler_grants_gov.query_opportunities import QueryOpportunities
from gofannon.simpler_grants_gov.query_opportunities_by_agency import QueryOpportunitiesByAgencyCode
from gofannon.simpler_grants_gov.query_by_funding_details import QueryByFundingDetails
from gofannon.simpler_grants_gov.query_by_applicant_eligibility import QueryByApplicantEligibility
from gofannon.simpler_grants_gov.query_by_award_criteria import QueryByAwardCriteria
from gofannon.simpler_grants_gov.query_by_dates import QueryByDates
from gofannon.simpler_grants_gov.query_by_assistance_listing import QueryByAssistanceListing
from gofannon.simpler_grants_gov.get_opportunity import GetOpportunity

2025-05-06 17:24:59,904 - gofannon.simpler_grants_gov.base - ERROR - Simpler Grants Gov API key is missing. Please set SIMPLER_GRANTS_API_KEY environment variable or pass api_key argument.
2025-05-06 17:24:59,907 - gofannon.simpler_grants_gov.query_opportunities.QueryOpportunities - WARNING - Simpler Grants Gov API key is missing. Please set SIMPLER_GRANTS_API_KEY environment variable or pass api_key argument. Tool execution will likely fail.
2025-05-06 17:24:59,909 - gofannon.simpler_grants_gov.base - ERROR - Simpler Grants Gov API key is missing. Please set SIMPLER_GRANTS_API_KEY environment variable or pass api_key argument.
2025-05-06 17:24:59,910 - gofannon.simpler_grants_gov.query_opportunities_by_agency.QueryOpportunitiesByAgencyCode - WARNING - Simpler Grants Gov API key is missing. Please set SIMPLER_GRANTS_API_KEY environment variable or pass api_key argument. Tool execution will likely fail.
2025-05-06 17:24:59,912 - gofannon.simpler_grants_gov.base - ERROR - Simpler Grants 

In [5]:
import logging

logging.getLogger('gofannon').setLevel(logging.DEBUG)

go = GetOpportunity(api_key=simpler_grants_api_key)
qo = QueryOpportunities(api_key=simpler_grants_api_key)
qobac = QueryOpportunitiesByAgencyCode(api_key=simpler_grants_api_key)
qbfd = QueryByFundingDetails(api_key=simpler_grants_api_key)
qbae = QueryByApplicantEligibility(api_key=simpler_grants_api_key)
qbac = QueryByAwardCriteria(api_key=simpler_grants_api_key)
qbda = QueryByDates(api_key=simpler_grants_api_key)
qbal = QueryByAssistanceListing(api_key=simpler_grants_api_key)

get_op_smol = go.export_to_smolagents()
qo_smol = qo.export_to_smolagents()
qobac_smol = qobac.export_to_smolagents()
qbfd_smol = qbfd.export_to_smolagents()
qbae_smol = qbae.export_to_smolagents()
qbac_smol = qbac.export_to_smolagents()
qbda_smol = qbda.export_to_smolagents()
qbal_smol = qbal.export_to_smolagents()

2025-05-06 17:24:59,942 - gofannon.simpler_grants_gov.query_opportunities.QueryOpportunities - DEBUG - Initialized QueryOpportunities tool
2025-05-06 17:24:59,944 - gofannon.simpler_grants_gov.query_opportunities.QueryOpportunities - DEBUG - Initialized QueryOpportunities with base_url: https://api.simpler.grants.gov and API key present
2025-05-06 17:24:59,945 - gofannon.simpler_grants_gov.query_opportunities_by_agency.QueryOpportunitiesByAgencyCode - DEBUG - Initialized QueryOpportunitiesByAgencyCode tool
2025-05-06 17:24:59,952 - gofannon.simpler_grants_gov.query_opportunities_by_agency.QueryOpportunitiesByAgencyCode - DEBUG - Initialized QueryOpportunitiesByAgencyCode with base_url: https://api.simpler.grants.gov and API key present
2025-05-06 17:24:59,953 - gofannon.simpler_grants_gov.query_by_funding_details.QueryByFundingDetails - DEBUG - Initialized QueryByFundingDetails tool
2025-05-06 17:24:59,956 - gofannon.simpler_grants_gov.query_by_funding_details.QueryByFundingDetails - D

## Create Agent

Next we will create an agent, powered by OpenAI's GPT-4o model. We will load our tools, and then query the model.

In [10]:
from smolagents import CodeAgent, ToolCallingAgent
from smolagents import OpenAIServerModel
model = OpenAIServerModel(
    model_id="gpt-4o",
    api_base="https://api.openai.com/v1",
    api_key=userdata.get('open_ai_key'),
    temperature= 0.2
)

tools=[qo_smol,
        qobac_smol,
        get_op_smol,
        qbfd_smol,
        qbae_smol,
        qbac_smol,
        qbda_smol,
        qbal_smol]

# agent = ToolCallingAgent(model=model, tools=tools)
agent = CodeAgent(model=model, tools=tools)


query = """
I have a small business that does AI related consulting work. And I'm partnered
with a university Help me find grants to pursue (that I have a fair
chance of winning). For each grant let me know what other types of organizations
I would need to partner with.
"""

output = agent.run(query, max_steps= 20)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I have a small business that does AI related consulting work. And I'm partnered                                 │
│ with a university Help me find grants to pursue (that I have a fair                                             │
│ chance of winning). For each grant let me know what other types of organizations                                │
│ I would need to partner with.                                                                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Searching for grant opportunities for small businesses partnered with universities                             
  applicant_types = ['small_businesses', 'public_and_state_controlled_institutions_of_higher_education']           
  query_text = "AI consulting"                                                                                     
  query_operator = "AND"                                                                                           
  items_per_page = 5                                                                                               
  page_number = 1                                                                                                  
  order_by = "relevancy"                                                                                           
  sort_direction = "descending"                                                                                    
  show_posted = True                                                                                               
  show_forecasted = False                                                                                          
  show_closed = False                                                                                              
  show_archived = False                                                                                            
                                                                                                                   
  grants = query_opportunities_by_applicant_eligibility(                                                           
      applicant_types=applicant_types,                                                                             
      requires_cost_sharing=False,                                                                                 
      query_text=query_text,                                                                                       
      query_operator=query_operator,                                                                               
      items_per_page=items_per_page,                                                                               
      page_number=page_number,                                                                                     
      order_by=order_by,                                                                                           
      sort_direction=sort_direction,                                                                               
      show_posted=show_posted,                                                                                     
      show_forecasted=show_forecasted,                                                                             
      show_closed=show_closed,                                                                                     
      show_archived=show_archived                                                                                  
  )                                                                                                                
                                                                                                                   
  print(grants)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

2025-05-06 17:29:46,543 - gofannon.simpler_grants_gov.query_by_applicant_eligibility.QueryByApplicantEligibility - INFO - Querying by applicant eligibility: types=['small_businesses', 'public_and_state_controlled_institutions_of_higher_education'], cost_sharing=False, query='AI consulting'
2025-05-06 17:29:46,544 - gofannon.simpler_grants_gov.query_by_applicant_eligibility.QueryByApplicantEligibility - DEBUG - Constructed API payload: {
  "pagination": {
    "page_offset": 1,
    "page_size": 5,
    "sort_order": [
      {
        "order_by": "relevancy",
        "sort_direction": "descending"
      }
    ]
  },
  "query": "AI consulting",
  "query_operator": "AND",
  "filters": {
    "opportunity_status": {
      "one_of": [
        "posted"
      ]
    },
    "applicant_type": {
      "one_of": [
        "small_businesses",
        "public_and_state_controlled_institutions_of_higher_education"
      ]
    },
    "is_cost_sharing": {
      "one_of": [
        false
      ]
    }
  }
}

Execution logs:
{"error": "Opportunity search API request failed: 422 Client Error: UNPROCESSABLE ENTITY for url: 
https://api.simpler.grants.gov/v1/opportunities/search", "success": false}

Out: None

...

[Step 7: Duration 13.73 seconds| Input tokens: 70,569 | Output tokens: 4,253]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Final summary of grant opportunities and potential partners                                                    
  final_summary = """                                                                                              
  1. Molecular Mechanisms of Combination Adjuvants (MMCA) (R01 Clinical Trial Not Allowed)                         
     - Agency: HHS-NIH11                                                                                           
     - Applicant Types: Includes small businesses, private institutions of higher education, public and state      
  institutions of higher education, and more.                                                                      
     - Summary: Supports research studies of vaccine adjuvants to understand their mechanisms and improve vaccine  
  design.                                                                                                          
                                                                                                                   
  2. Asthma and Allergic Diseases Cooperative Research Centers (U19 Clinical Trial Optional)                       
     - Agency: HHS-NIH11                                                                                           
     - Applicant Types: Includes small businesses, private institutions of higher education, public and state      
  institutions of higher education, and more.                                                                      
     - Summary: Invites applications to participate in research centers focused on asthma and allergic diseases,   
  integrating clinical and translational research.                                                                 
                                                                                                                   
  3. Examining the Impact of Artificial Intelligence (AI) on Healthcare Safety (R18)                               
     - Agency: HHS-AHRQ                                                                                            
     - Applicant Types: Includes small businesses, private institutions of higher education, public and state      
  institutions of higher education, and more.                                                                      
     - Summary: Supports applications that explore the impact of AI on healthcare safety, focusing on patient      
  safety and implementation of AI systems.                                                                         
                                                                                                                   
  For each of these opportunities, partnering with educational institutions such as universities (both private     
  and public) could strengthen your application. Additionally, consider collaborating with other eligible          
  applicant types mentioned for each grant to enhance your proposal's competitiveness.                             
  """                                                                                                              
                                                                                                                   
  final_answer(final_summary)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
1. Molecular Mechanisms of Combination Adjuvants (MMCA) (R01 Clinical Trial Not Allowed)
   - Agency: HHS-NIH11
   - Applicant Types: Includes small businesses, private institutions of higher education, public and state 
institutions of higher education, and more.
   - Summary: Supports research studies of vaccine adjuvants to understand their mechanisms and improve vaccine 
design.

2. Asthma and Allergic Diseases Cooperative Research Centers (U19 Clinical Trial Optional)
   - Agency: HHS-NIH11
   - Applicant Types: Includes small businesses, private institutions of higher education, public and state 
institutions of higher education, and more.
   - Summary: Invites applications to participate in research centers focused on asthma and allergic diseases, 
integrating clinical and translational research.

3. Examining the Impact of Artificial Intelligence (AI) on Healthcare Safety (R18)
   - Agency: HHS-AHRQ
   - Applicant Types: Includes small businesses, private institutions of higher education, public and state 
institutions of higher education, and more.
   - Summary: Supports applications that explore the impact of AI on healthcare safety, focusing on patient safety 
and implementation of AI systems.

For each of these opportunities, partnering with educational institutions such as universities (both private and 
public) could strengthen your application. Additionally, consider collaborating with other eligible applicant types
mentioned for each grant to enhance your proposal's competitiveness.

[Step 8: Duration 8.43 seconds| Input tokens: 90,849 | Output tokens: 4,613]